# Client

I've included the example image client here(you'll need the densenet_onnx from the Triton repo to run it) but hopefully this will serve as a solid baseline for building up our client structure.

In [1]:
!pip install tritonclient[all]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 8.4 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 251 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 11.5 MB/s eta 0:00:01


In [1]:
!wget https://raw.githubusercontent.com/Wolfffff/gpuhackathon-sleap/main/triton/triton_utils.py

from triton_utils import parse_model
from triton_utils import UserData
from triton_utils import requestGenerator

import numpy as np
import cv2
import tensorflow as tf
import tritonclient.grpc as grpcclient

--2021-05-26 23:05:15--  https://raw.githubusercontent.com/Wolfffff/gpuhackathon-sleap/main/triton/triton_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4555 (4.4K) [text/plain]
Saving to: ‘triton_utils.py.1’

triton_utils.py.1   100%[===================>]   4.45K  --.-KB/s    in 0s      

2021-05-26 23:05:15 (10.5 MB/s) - ‘triton_utils.py.1’ saved [4555/4555]



ModuleNotFoundError: No module named 'attrdict'

In [3]:
def read_frames(video_path, fidxs=None, grayscale=True):
    """Read frames from a video file.
    
    Args:
        video_path: Path to MP4
        fidxs: List of frame indices or None to read all frames (default: None)
        grayscale: Keep only one channel of the images (default: True)
    
    Returns:
        Loaded images in array of shape (n_frames, height, width, channels) and dtype uint8.
    """
    vr = cv2.VideoCapture(video_path)
    if fidxs is None:
        fidxs = np.arange(vr.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for fidx in fidxs:
        vr.set(cv2.CAP_PROP_POS_FRAMES, fidx)
        img = vr.read()[1]
        if grayscale:
            img = img[:, :, [0]]
        frames.append(img)
    return np.stack(frames, axis=0)

In [4]:
test_data = "../tensorrt/data/190719_090330_wt_18159206_rig1.2@15000-17560.mp4"
image_data = read_frames(test_data,np.arange(16)).astype('float32')


In [5]:
# Resize depending on the model we're actually going to use -- full for bu, 0.5 scale for centroid
image_data = tf.image.resize(image_data,size=(512,512)).numpy()

In [6]:

url = 'localhost:8001'
model_name = "centroid_savedmodel"
model_version="1"
protocol = 'grpc'
triton_client = grpcclient.InferenceServerClient(
                url=url)

In [7]:
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)

model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

model_config = model_config.config

max_batch_size, input_name, output_name, c, h, w, format, dtype = parse_model(model_metadata, model_config)

# Fix output names for when we have multiple
output_names = [model.name for model in model_metadata.outputs]

In [8]:
requests = []
responses = []
result_filenames = []
request_ids = []
image_idx = 0
last_request = False
user_data = UserData()

sent_count = 0

In [9]:
# Pair request generator 
for inputs, outputs,model_name, model_version in requestGenerator(
        image_data, input_name, output_names, dtype, protocol,model_name,model_version):
    sent_count += 1
    responses.append(triton_client.infer(model_name,
                                inputs,
                                request_id=str(sent_count),
                                model_version=model_version,
                                outputs=outputs))

In [2]:
import matplotlib.pyplot as plt

plt.imshow(responses[0].as_numpy(output_names[0])[15])

NameError: name 'responses' is not defined